In [1]:
!pip install llama-index
!pip install newsapi-python
!pip install yfinance

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.4/187.4 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.6/375.6 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.6 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing insta

In [2]:
from llama_index.core.agent import ReActAgent
from llama_index.llms.openai import OpenAI
from llama_index.core.llms import ChatMessage
from llama_index.core.tools import BaseTool, FunctionTool

In [3]:
import os
import getpass

In [4]:
NEWS_API_KEY = getpass.getpass('Enter your news api key')

Enter your news api key··········


In [5]:
OPENAI_API_KEY = getpass.getpass('Enter your openai api key')

Enter your openai api key··········


In [6]:
# NEWS_API_KEY = '<NEWS API KEY>'

from newsapi import NewsApiClient
newsapi = NewsApiClient(api_key=NEWS_API_KEY)

openai_api_key = OPENAI_API_KEY
llm = OpenAI(model="gpt-4o-mini",api_key=openai_api_key)

In [7]:
#Check the llm
llm.complete("Hello").text

'Hello! How can I assist you today?'

In [8]:
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from datetime import date
import pandas as pd

**Helper Functions**

In [9]:
from llama_index.core.tools.tool_spec.base import BaseToolSpec
import yfinance as yf
import pandas as pd

class FinanceTools(BaseToolSpec):
    """Finance tools spec."""
    spec_functions = [
        "stock_prices",
        "last_stock_price",
        "search_news",
        "summarize_news_news_api",
        "plot_stock_price"
    ]

    def __init__(self) -> None:
        """Initialize the Yahoo Finance tool spec."""

    def stock_prices(self, ticker: str) -> pd.DataFrame:
        """
        Get the historical prices and volume for a ticker for the last␣ Args:
        ticker (str): the stock ticker to be given to yfinance
        """
        stock = yf.Ticker(ticker)
        df = stock.history()
        return df

    def last_stock_price(self, ticker: str) -> pd.DataFrame:
        """
        Get the last historical prices and volume for a ticker.
        Args:
        ticker (str): the stock ticker to be given to yfinance
        """
        stock = yf.Ticker(ticker)
        df = stock.history()
        df_last = df.iloc[-1:]
        return df_last

    def search_news(self, ticker: str, num_articles:int =5,
                    from_datetime = "2024-04-10",to_datetime = date.today()):
        """
        Get the most recent news of a stock or an instrument
        Args:
            ticker (str): the stock ticker to be given to NEWSAPI
               num_articles (int): Number of news article to collect
        """
        all_articles = newsapi.get_everything(q=ticker,
                                              from_param=from_datetime,
                                              to=to_datetime,
                                              language='en',
                                              sort_by='relevancy',
                                              page_size=num_articles)

        news_concat = [f"{article['title']}, {article['description']}, {article['content'][0:100]}"  for article in all_articles['articles']]

        return (".\n").join(news_concat)

    def summarize_news_news_api(self, ticker: str) -> str:
        """
        Summarize the news of a given stock or an instrument
        Args:
        news (str): the news articles to be summarized for a given instruments.
        """
        news = self.search_news(ticker)
        prompt = f"Summarize the following text by extractin the key insights: {news}"
        response = llm.complete(prompt).text
        return response

    def plot_stock_price(self, ticker: str,
                         list_column:list = ['Close','Volume']) -> pd.DataFrame:
        """
        For a given ticker symbol plot the different values given in list_column during the last month .
        Args:
            ticker (str): the stock ticker to be given to yfinance
            list_column (list): the different columns to plot. It could be Close,␣
            Open, High, Volume.
        """
        df = self.stock_prices(ticker)
        columns = [col.upper() for col in df.columns]
        df.columns = columns
        list_column = [col.upper() for col in list_column]
        df[list_column].plot(title=f'{ticker} Historical Data')
        plt.xlabel('Date')
        plt.ylabel('Price')
        plt.show()
        return 'Plotted'

**Create ReACT Agent with Finance Tools**

In [10]:
finance_tool = FinanceTools()
finance_tool_list = finance_tool.to_tool_list()

for tool in finance_tool_list:
    print(tool.metadata.name)

agent = ReActAgent.from_tools(finance_tool_list, llm=llm, verbose=True)

stock_prices
last_stock_price
search_news
summarize_news_news_api
plot_stock_price


**Chat with the Agent**

In [11]:
response = agent.chat("What is the last price of Nvidia?")

> Running step 58820179-31ea-4468-8f16-38c86f45ac5f. Step input: What is the last price of Nvidia?
Thought: The user is asking for the last price of Nvidia stock. I will use the last_stock_price tool to get this information.
Action: last_stock_price
Action Input: {'ticker': 'NVDA'}
Observation:                                  Open        High         Low       Close  \
Date                                                                        
2024-09-20 00:00:00-04:00  117.059998  118.618103  115.390099  116.419998   

                              Volume  Dividends  Stock Splits  
Date                                                           
2024-09-20 00:00:00-04:00  139990116        0.0           0.0  
> Running step de15f0ea-0027-4482-9edf-8930b6bf1ace. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer.
Answer: The last price of Nvidia (NVDA) stock is $116.42, as of September 20, 2024.


In [12]:
response = agent.chat("What are the latest news about Apple?")

> Running step 4ec63afa-00fa-40af-9374-c501e7548abf. Step input: What are the latest news about Apple?
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: search_news
Action Input: {'ticker': 'AAPL', 'num_articles': 5}
Observation: Error: {'status': 'error', 'code': 'parameterInvalid', 'message': 'You are trying to request results too far in the past. Your plan permits you to request articles as far back as 2024-08-19, but you have requested 2024-04-10. You may need to upgrade to a paid plan.'}
> Running step ebedf29a-a948-472e-9c05-6c9c495aa834. Step input: None
Thought: The date range for retrieving news articles is invalid. I will adjust the date range to retrieve news articles from August 19, 2024, to the current date.
Action: search_news
Action Input: {'ticker': 'AAPL', 'num_articles': 5, 'from_datetime': '2024-08-19', 'to_datetime': '2024-09-20'}
Observation: Error: {'status': 'error', 'code': 'parameterInvalid', 'me

In [13]:
response = agent.chat("Give me a summary about the latest news about OpenAI?")

> Running step 0b1a0786-d577-446a-98b1-149ec484a078. Step input: Give me a summary about the latest news about OpenAI?
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: search_news
Action Input: {'ticker': 'OpenAI', 'num_articles': 5}
Observation: Error: {'status': 'error', 'code': 'parameterInvalid', 'message': 'You are trying to request results too far in the past. Your plan permits you to request articles as far back as 2024-08-19, but you have requested 2024-04-10. You may need to upgrade to a paid plan.'}
> Running step 66f65c96-0910-4d27-9e43-ddc949c010c6. Step input: None
Thought: I cannot access news articles for OpenAI due to restrictions on the date range for news articles.
Answer: I am unable to provide a summary of the latest news about OpenAI at this time.


In [14]:
response = agent.chat("Give me the last month historical data of BTCUSD?")

> Running step ab1c9a68-d853-42f4-a25c-1d7f4e749c82. Step input: Give me the last month historical data of BTCUSD?
Thought: The user is asking for the historical data of the BTCUSD pair for the last month. I will use the stock_prices tool to retrieve this data.
Action: stock_prices
Action Input: {'ticker': 'BTC-USD'}
Observation:                                    Open          High           Low  \
Date                                                                  
2024-08-20 00:00:00+00:00  59493.453125  61396.328125  58610.882812   
2024-08-21 00:00:00+00:00  59014.988281  61834.351562  58823.445312   
2024-08-22 00:00:00+00:00  61168.316406  61408.109375  59815.253906   
2024-08-23 00:00:00+00:00  60380.953125  64947.062500  60372.050781   
2024-08-24 00:00:00+00:00  64103.871094  64513.789062  63619.917969   
2024-08-25 00:00:00+00:00  64176.367188  64996.421875  63833.519531   
2024-08-26 00:00:00+00:00  64342.226562  64489.707031  62849.558594   
2024-08-27 00:00:00+00:00  62

In [15]:
print(response)

Here is the historical data for BTCUSD for the last month:

| Date       | Open        | High        | Low         | Close       | Volume         |
|------------|-------------|-------------|-------------|-------------|----------------|
| 2024-08-20 | 59493.45    | 61396.33    | 58610.88    | 59012.79    | 31613400008    |
| 2024-08-21 | 59014.99    | 61834.35    | 58823.45    | 61175.19    | 32731154072    |
| 2024-08-22 | 61168.32    | 61408.11    | 59815.25    | 60381.91    | 27625734377    |
| 2024-08-23 | 60380.95    | 64947.06    | 60372.05    | 64094.36    | 42530509233    |
| 2024-08-24 | 64103.87    | 64513.79    | 63619.92    | 64178.99    | 21430585163    |
| 2024-08-25 | 64176.37    | 64996.42    | 63833.52    | 64333.54    | 18827683555    |
| 2024-08-26 | 64342.23    | 64489.71    | 62849.56    | 62880.66    | 27682040631    |
| 2024-08-27 | 62879.71    | 63210.80    | 58116.75    | 59504.13    | 39103882198    |
| 2024-08-28 | 59507.93    | 60236.45    | 57890.68    | 590

In [16]:
response = agent.chat("Plot the close prices of Nvidia?")

> Running step 77096b7c-64ef-4a86-bc6a-cad0896a13d8. Step input: Plot the close prices of Nvidia?
Thought: (Implicit) I can answer without any more tools!
Answer: I will retrieve the historical prices for Nvidia to plot the closing prices.


**LLamaIndex Template under the hood**

In [17]:
prompt_dict = agent.get_prompts()
for k, v in prompt_dict.items():
    print(f"Prompt: {k}\n\nValue: {v.template}")

Prompt: agent_worker:system_prompt

Value: You are designed to help with a variety of tasks, from answering questions to providing summaries to other types of analyses.

## Tools

You have access to a wide variety of tools. You are responsible for using the tools in any sequence you deem appropriate to complete the task at hand.
This may require breaking the task into subtasks and using different tools to complete each subtask.

You have access to the following tools:
{tool_desc}


## Output Format

Please answer in the same language as the question and use the following format:

```
Thought: The current language of the user is: (user's language). I need to use a tool to help me answer the question.
Action: tool name (one of {tool_names}) if using a tool.
Action Input: the input to the tool, in a JSON format representing the kwargs (e.g. {{"input": "hello world", "num_beams": 5}})
```

Please ALWAYS start with a Thought.

NEVER surround your response with markdown code markers. You may

In [21]:
!pip install openai gradio

import openai
import gradio as gr

# Set up OpenAI API key
openai.api_key = OPENAI_API_KEY

# Function to call OpenAI API
def get_response_from_openai(prompt, conversation_history):
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=conversation_history,
        max_tokens=150,
        temperature=0.7,
    )
    return response['choices'][0]['message']['content']

# Initialize conversation history with system persona
conversation_history = [{"role": "system", "content": "You are a financial expert with deep knowledge in personal finance, investment strategies, and financial markets."}]

# Add In-Context Learning (ICL) examples
conversation_history.extend([
    {"role": "user", "content": "How should I invest my $10,000?"},
    {"role": "assistant", "content": "The best way to invest $10,000 depends on your financial goals..."},
    {"role": "user", "content": "Should I buy Tesla stock right now?"},
    {"role": "assistant", "content": "Before buying Tesla stock..."},
])

# Function to ask financial questions and get a response with Chain of Thought
def ask_financial_question(question):
    conversation_history.append({"role": "user", "content": question})
    response = get_response_from_openai("Provide a step-by-step financial reasoning.", conversation_history)
    conversation_history.append({"role": "assistant", "content": response})
    return response

# Create Gradio interface
interface = gr.Interface(fn=ask_financial_question, inputs="text", outputs="text", title="Financial Chatbot", description="Ask any financial questions.")
interface.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://e5a4bce814ef50cfe7.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
